# 詞神林夕養成計畫(keras)

![md_images](../Images/charrnn.png)

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import os


Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF


def get_session(gpu_fraction=0.3):
    """
    This function is to allocate GPU memory a specific fraction
    Assume that you have 6GB of GPU memory and want to allocate ~2GB
    """
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.4))  # using 40% of total GPU Memory

In [3]:
with io.open('lingxi.txt', encoding='utf-8-sig') as f:
    text = f.read().lower()
print('corpus length:', len(text))


corpus length: 52647


In [4]:
#把每個字去重複
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 2114


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(x.shape)

nb sequences: 17536
Vectorization...
(17536, 40, 2114)


In [6]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))


optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

if os.path.exists('Models/lingxi_keras.h5'):
    model.load_weights('Models/lingxi_keras.h5')
    print('load model')
print (model.summary())

Build model...
load model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1148416   
_________________________________________________________________
dense_1 (Dense)              (None, 2114)              272706    
_________________________________________________________________
activation_1 (Activation)    (None, 2114)              0         
Total params: 1,421,122
Trainable params: 1,421,122
Non-trainable params: 0
_________________________________________________________________
None


In [7]:

def sample(preds, temperature=1.0):
	# helper function to sample an index from a probability array
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)


def on_epoch_end(epoch, logs):
	# Function invoked at end of each epoch. Prints generated text.
	print()
	print('----- 第Epoch: %d後自動寫詞' % epoch)
	model.save_weights('Models/lingxi_keras.h5',True)
	start_index = random.randint(0, len(text) - maxlen - 1)
	for diversity in [0.5, 0.8,1.0]:
		print('----- diversity:', diversity)
		
		generated = ''
		sentence = text[start_index: start_index + maxlen]
		generated += sentence
		print('----- 根據以下詞彙發想: "' + sentence + '"')
		sys.stdout.write(generated)
		
		for i in range(400):
			x_pred = np.zeros((1, maxlen, len(chars)))
			for t, char in enumerate(sentence):
				x_pred[0, t, char_indices[char]] = 1.
			
			preds = model.predict(x_pred, verbose=0)[0]
			next_index = sample(preds, diversity)
			next_char = indices_char[next_index]
			
			generated += next_char
			sentence = sentence[1:] + next_char
			
			sys.stdout.write(next_char)
			sys.stdout.flush()
		print()


In [8]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])



Epoch 1/5
17536/17536 [==============================] - 18s 1ms/step - loss: 1.1789 - acc: 0.8075

----- 第Epoch: 0後自動寫詞
----- diversity: 0.5
----- 根據以下詞彙發想: " 賜我反應
有你陪伴氣氛 總有保証
娛樂我是你本領
唯獨你當我最快樂時 替我高興"
 賜我反應
有你陪伴氣氛 總有保証
娛樂我是你本領
唯獨你當我最快樂時 替我高興
當我的我也不想誰
我不想追他
曾經可 大個女之間 像你見不見
情情那麼不 你對我說說話 我知我的我
也許這對我
我要你大個在乎
我那快樂是是何日知我
在你前裡會有甚麼無戀
那人b各 各自上何去
何又會不會說 愛情有你 不知道
如果你愛情感覺到
若你仍然發戀愛情感覺到你
是否可怕沉命看見清醒反生走
不再你會更早心我自己
未紅最愛  越都相感情
你要你的我只有好
因有愛 誰能講不如說話
對你說你話在乎
前熱戀u得 沒法相處
再不起你 我愛誰 我要心的人
〔可有誰會說 在誰裡有在自地
誰如在留下著你你受的得 你我最好的他
可能跟你同有了誰就好
用我這用我的不心 我的心怪你笑
可不可 我多人樂人
我要風樣一對 是你的心在 並演你千心在我跌早重
也許假夜〔仍然放開
還多多少我不能愛誰 我要是你
是是一生 是是誰是誰 誰能知
男知是是我自氣
愛我會 天 誰會知在孤孤單不到
為頭眼睛看 不只你愛情對

----- diversity: 0.8
----- 根據以下詞彙發想: " 賜我反應
有你陪伴氣氛 總有保証
娛樂我是你本領
唯獨你當我最快樂時 替我高興"
 賜我反應
有你陪伴氣氛 總有保証
娛樂我是你本領
唯獨你當我最快樂時 替我高興等不一再一和
是你的對 你是好  仍是是是是是友不知
訴你我說留 愛情 將是這種話下說我說話 我多誰能要你你
知道 就黑分念
天生美麗
留在路人要情令我 假使有方感
如重盡不會相信
從此相愛戀愛你 恨日後是是誰 時可分享吧錯
u電
等戀快樂大是是我得在這最夢
用一點永遠不說一場真
是是 告別有甚麼也不忘
也可生這故事 情活你的心事
難可以跟你再未落來 〔感覺不住
當天不大你想在跳
如果想要停情 真不不怕太閃福 只怕曾錯
愛情人 多得盡苦軟
要再多大箏背已 痛苦中當風機
機